# Concatenating Tables with Set-Like Operations in `pyspark`

Now let's look at combining tables with `union`, `intersect`, and `except` in `pyspark`.

## Example - Auto Sales in Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').config('spark.driver.host', 'localhost').getOrCreate()

In [2]:
from more_pyspark import to_pandas
sales_aprk = spark.read.csv("./data/auto_sales_apr.csv",  header=True, inferSchema=True)
sales_aprk.collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [3]:
sales_mayk = spark.read.csv("./data/auto_sales_may.csv",  header=True, inferSchema=True)
sales_mayk.collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


## `UNION ALL` in `pyspark`

Both `union` and `unionAll` area actually `UNION ALL`

# 

In [4]:
(sales_aprk
 .union(sales_mayk)
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9
4,0,Ann,22,18,15,12
5,1,Bob,20,14,6,24
6,2,Yolanda,19,10,28,17
7,3,Xerxes,11,27,17,9


## `UNION/UNION DISTINCT` in `pyspark`

Use `distinct` to get the usual `UNION/UNION DISTINCT`

In [5]:
(sales_aprk
 .union(sales_mayk)
 .distinct()
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,2,Yolanda,19,8,32,15
1,3,Xerxes,12,23,18,9
2,1,Bob,20,14,6,24
3,2,Yolanda,19,10,28,17
4,3,Xerxes,11,27,17,9
5,1,Bob,19,12,17,20
6,0,Ann,22,18,15,12


## Adding a `month` column

As mentioned before, we really should add a month column here. Note that we need to use `lit` to add a *literal constant*

In [6]:
from pyspark.sql.functions import lit

(sales_aprk.withColumn('month', lit('April'))
 .union(sales_mayk.withColumn('month', lit('May')))
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,April
1,1,Bob,19,12,17,20,April
2,2,Yolanda,19,8,32,15,April
3,3,Xerxes,12,23,18,9,April
4,0,Ann,22,18,15,12,May
5,1,Bob,20,14,6,24,May
6,2,Yolanda,19,10,28,17,May
7,3,Xerxes,11,27,17,9,May


## Performing `INTERSECT`

Note that `intersect` and `intersectAll` are synonymous.

In [7]:
sales_aprk.intersect(sales_mayk).collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


## Performing a set difference with `exceptAll`

In [8]:
sales_aprk.exceptAll(sales_mayk).collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,1,Bob,19,12,17,20
1,2,Yolanda,19,8,32,15
2,3,Xerxes,12,23,18,9


## <font color="red"> Exercise 2 </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 `pyspark` dataframes into a `dict` with keys equal to the month name and values containing the corresponding data frame.
4. Use a dictionary comprehension to add a month column to each `df`.
5. Use the accumulator pattern and the `union` method to combine these 6 data frames into one combined `pysaprk df`
6. Inspect the head and compute the number of rows (use the `count` method)

In [9]:
# Your code here